# Timeseries generation with HYP3_ISCE2

Single Bust for now Submit 2 interferograms as 'batch job'
https://hyp3-docs.asf.alaska.edu/guides/burst_insar_product_guide/

In [1]:
import hyp3_sdk as sdk
import asf_search as asf
import geopandas as gpd
import pandas as pd
import os
import copy

In [2]:
hyp3 = sdk.HyP3() # 'https://hyp3-api.asf.alaska.edu'
print('User:{user_id} Quota:{quota}'.format(**hyp3.my_info()))

User:scottyhq Quota:{'max_jobs_per_month': 1000, 'remaining': 657}


## Search for acquisitions

In [3]:
# https://geojson.io/#new&map=15.23/47.654452/-122.303174
gfa = gpd.GeoDataFrame.from_features( {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          -122.29994319751066,
          47.657472535245574
        ],
        "type": "Point"
      }
    }
  ]
},
 crs=4326   
)

In [5]:
# HyP3Error: <Response [400]> Only VV and HH polarizations are currently supported, got: VH

results = asf.search(
                    platform=[asf.PLATFORM.SENTINEL1],
                    processingLevel=asf.BURST,
                    beamMode=asf.BEAMMODE.IW,
                    #relativeOrbit=relorb,
                    polarization='VV',
                    flightDirection='ASCENDING',
                    intersectsWith=str(gfa.geometry.values[0]),
                    start='2019-06-01',
                    end='2023-09-01',
                    season=[180,270], #first,last day of year June~30*6=180
                    )
gf = gpd.GeoDataFrame.from_features(results.geojson(), crs=4326)
print(len(gf))
# Useful to fine burstIDs of interest

68


In [6]:
gf['burstId'] = gf.burst.str['fullBurstID']
gf.groupby('burstId').sceneName.count()

burstId
064_135602_IW1    33
137_292399_IW2    35
Name: sceneName, dtype: int64

In [8]:
# Pick a single burst ID to work with
burst = '064_135602_IW1'

In [9]:
# Look at timeline of acquisition
gfb = gf[gf.burstId == burst]
gfb.loc[:,'datetime'] = pd.to_datetime(gfb.startTime)

/home/jovyan/.local/envs/hyp3-isce2/lib/python3.11/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [10]:
#gfb.plot.scatter(x='datetime', y=1);

In [11]:
subset = gfb.set_index('datetime')
subset = subset[subset.index.month == 8] #best odds for good weather 

# Lots of selection opytion - random, first, last,e tc
subset = subset.groupby(subset.index.year).last()
subset.fileID.to_list()

['S1_135602_IW1_20190803T015332_VV_D562-BURST',
 'S1_135602_IW1_20200809T015339_VV_CAFE-BURST',
 'S1_135602_IW1_20210804T015344_VV_7959-BURST',
 'S1_135602_IW1_20220805T015433_VV_1512-BURST',
 'S1_135602_IW1_20230812T015437_VV_8FF9-BURST']

In [12]:
subset['date'] = pd.to_datetime(subset.startTime) #.dt.strftime('%Y%m%d')

In [13]:
job_definition = {
  "name": None,
  "job_type": "INSAR_ISCE_BURST",
  "job_parameters": {
      "granules": [None,None],
      "apply_water_mask": True, #default=False not enabled?
      "looks": "5x1", # '20x4', '10x2', '5x1' default=20x4
  },
}
job_definition

{'name': None,
 'job_type': 'INSAR_ISCE_BURST',
 'job_parameters': {'granules': [None, None],
  'apply_water_mask': True,
  'looks': '5x1'}}

In [14]:
# Create 1-year timespan interferograms
jobs = [] # List of job dictionaries

for i in range(len(subset)-1):
    jobdef = copy.deepcopy(job_definition) 
    
    ref = subset.iloc[i]
    sec = subset.iloc[i+1]
    jobname = f"{burst}_{ref.date.strftime('%Y%m%d')}_{sec.date.strftime('%Y%m%d')}_5rlks"
    
    jobdef['name'] = jobname
    jobdef['job_parameters']['granules'] = [ref.fileID,sec.fileID]
    jobs.append(jobdef)

In [15]:
# NOTE: should complete in ~20minutes
jobs

[{'name': '064_135602_IW1_20190803_20200809_5rlks',
  'job_type': 'INSAR_ISCE_BURST',
  'job_parameters': {'granules': ['S1_135602_IW1_20190803T015332_VV_D562-BURST',
    'S1_135602_IW1_20200809T015339_VV_CAFE-BURST'],
   'apply_water_mask': True,
   'looks': '5x1'}},
 {'name': '064_135602_IW1_20200809_20210804_5rlks',
  'job_type': 'INSAR_ISCE_BURST',
  'job_parameters': {'granules': ['S1_135602_IW1_20200809T015339_VV_CAFE-BURST',
    'S1_135602_IW1_20210804T015344_VV_7959-BURST'],
   'apply_water_mask': True,
   'looks': '5x1'}},
 {'name': '064_135602_IW1_20210804_20220805_5rlks',
  'job_type': 'INSAR_ISCE_BURST',
  'job_parameters': {'granules': ['S1_135602_IW1_20210804T015344_VV_7959-BURST',
    'S1_135602_IW1_20220805T015433_VV_1512-BURST'],
   'apply_water_mask': True,
   'looks': '5x1'}},
 {'name': '064_135602_IW1_20220805_20230812_5rlks',
  'job_type': 'INSAR_ISCE_BURST',
  'job_parameters': {'granules': ['S1_135602_IW1_20220805T015433_VV_1512-BURST',
    'S1_135602_IW1_2023081

In [16]:
jobs = hyp3.submit_prepared_jobs(jobs)

In [17]:
b = hyp3.find_jobs(job_type='INSAR_ISCE_BURST')

In [18]:
df = pd.DataFrame([x.to_dict() for x in b])
df

,job_id,job_type,request_time,status_code,user_id,name,job_parameters,files,logs,browse_images,thumbnail_images,expiration_time,processing_times
0,952e6f9b-dfe4-46d6-b022-8f313150d994,INSAR_ISCE_BURST,2023-12-09T05:57:41+00:00,PENDING,scottyhq,064_135602_IW1_20220805_20230812_5rlks,"{'apply_water_mask': True, 'granules': ['S1_13...",NaN,NaN,NaN,NaN,NaN,NaN
1,a5de36d9-dc90-4417-965c-73136ff873ee,INSAR_ISCE_BURST,2023-12-09T05:57:41+00:00,PENDING,scottyhq,064_135602_IW1_20200809_20210804_5rlks,"{'apply_water_mask': True, 'granules': ['S1_13...",NaN,NaN,NaN,NaN,NaN,NaN
2,13ce7a4c-23fa-415c-8693-53cf543854a7,INSAR_ISCE_BURST,2023-12-09T05:57:41+00:00,PENDING,scottyhq,064_135602_IW1_20210804_20220805_5rlks,"{'apply_water_mask': True, 'granules': ['S1_13...",NaN,NaN,NaN,NaN,NaN,NaN
3,f8a1bcfd-f435-433b-8f6c-a943ba89400b,INSAR_ISCE_BURST,2023-12-09T05:57:41+00:00,PENDING,scottyhq,064_135602_IW1_20190803_20200809_5rlks,"{'apply_water_mask': True, 'granules': ['S1_13...",NaN,NaN,NaN,NaN,NaN,NaN
4,a226e75f-8b59-404a-8bb2-de78058645f9,INSAR_ISCE_BURST,2023-12-09T05:09:19+00:00,SUCCEEDED,scottyhq,064_135602_IW1_20201008_20200926_5rlks,"{'apply_water_mask': True, 'granules': ['S1_13...",[{'filename': 'S1_135602_IW1_20200926_20201008...,[],[https://d3gm2hf49xd6jj.cloudfront.net/a226e75...,[https://d3gm2hf49xd6jj.cloudfront.net/a226e75...,2023-12-24T00:00:00+00:00,[1298.542]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,e603568e-dcc5-4340-9ee2-8037ee2acda4,INSAR_ISCE_BURST,2023-12-07T22:09:23+00:00,SUCCEEDED,scottyhq,115_245655_IW3_20220703_20230710_5rlks,{'granules': ['S1_245655_IW3_20220703T141349_V...,[{'filename': 'S1_245655_IW3_20220703_20230710...,[],[https://d3gm2hf49xd6jj.cloudfront.net/e603568...,[https://d3gm2hf49xd6jj.cloudfront.net/e603568...,2023-12-22T00:00:00+00:00,[1224.588]
343,e91d2861-a856-41f3-b40a-e923e9d2ad61,INSAR_ISCE_BURST,2023-12-07T22:09:23+00:00,SUCCEEDED,scottyhq,115_245655_IW3_20220703_20230710_5rlks,{'granules': ['S1_245655_IW3_20220703T141349_V...,[{'filename': 'S1_245655_IW3_20220703_20230710...,[],[https://d3gm2hf49xd6jj.cloudfront.net/e91d286...,[https://d3gm2hf49xd6jj.cloudfront.net/e91d286...,2023-12-22T00:00:00+00:00,[1224.607]
344,5a804b93-29b6-4859-a21f-b19de7b45b79,INSAR_ISCE_BURST,2023-12-07T21:34:08+00:00,SUCCEEDED,scottyhq,S1_245655_IW3_20220727_20230722_10rlks,"{'apply_water_mask': True, 'granules': ['S1_24...",[{'filename': 'S1_245655_IW3_20220727_20230722...,[],[https://d3gm2hf49xd6jj.cloudfront.net/5a804b9...,[https://d3gm2hf49xd6jj.cloudfront.net/5a804b9...,2023-12-22T00:00:00+00:00,[569.17]
345,7c2b6d97-219b-4843-bb9a-a9bb0935fbb3,INSAR_ISCE_BURST,2023-12-07T21:28:50+00:00,SUCCEEDED,scottyhq,S1_245655_IW3-test2,{'granules': ['S1_245655_IW3_20220727T141350_V...,[{'filename': 'S1_245655_IW3_20220727_20230722...,[],[https://d3gm2hf49xd6jj.cloudfront.net/7c2b6d9...,[https://d3gm2hf49xd6jj.cloudfront.net/7c2b6d9...,2023-12-22T00:00:00+00:00,[1111.138]
